<a href="https://colab.research.google.com/github/tina287/fianceHomework/blob/main/%E9%87%91%E8%9E%8D%E5%A4%A7%E6%95%B8%E6%93%9A_%E5%9F%BA%E5%9B%A0%E6%BC%94%E7%AE%97%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# 簡化版基因演算法：最佳化投資組合

# 匯入必要的套件
import yfinance as yf
import numpy as np
import random

# 固定隨機種子
np.random.seed(42)
random.seed(42)

# 年化報酬與風險計算
def calculate_annual_metrics(data):
    years = (data.index[-1] - data.index[0]).days / 365.0
    start, end = data.iloc[0], data.iloc[-1]
    annual_return = (end / start) ** (1 / years) - 1
    daily_returns = data.pct_change().dropna()
    annual_risk = daily_returns.std() * np.sqrt(len(daily_returns))
    return annual_return, annual_risk

# 取得股票資料
def fetch_stock_data(symbols, start_date, end_date):
    returns, risks = [], []
    for symbol in symbols:
        data = yf.download(f"{symbol}.TW", start=start_date, end=end_date)["Adj Close"]
        ret, risk = calculate_annual_metrics(data)
        returns.append(ret)
        risks.append(risk)
    return np.array(returns), np.array(risks)

# 適應函數
def fitness(chromosome, stock_returns, stock_risks):
    portfolio_return = np.sum(chromosome * stock_returns)
    portfolio_risk = np.sqrt(np.sum((chromosome * stock_risks) ** 2))
    sharpe_ratio = portfolio_return / portfolio_risk if portfolio_risk > 0 else 0
    return 0.8 * sharpe_ratio - 0.006 * np.sum(chromosome)

# 初始化、選擇、交配、突變
def initialize_population(size, num_stocks):
    return [np.random.randint(0, 2, num_stocks) for _ in range(size)]

def select(population, fitness_scores):
    probs = (np.array(fitness_scores) - min(fitness_scores) + 1e-6)
    probs /= probs.sum()
    return population[np.random.choice(len(population), p=probs)]

def crossover(parent1, parent2):
    point = np.random.randint(1, len(parent1))
    return np.concatenate((parent1[:point], parent2[point:])), np.concatenate((parent2[:point], parent1[point:]))

def mutate(chromosome, rate):
    for i in range(len(chromosome)):
        if random.random() < rate:
            chromosome[i] = 1 - chromosome[i]
    return chromosome

# GA 主程式
def genetic_algorithm(stock_returns, stock_risks, num_stocks, population_size=20, generations=200, mutation_rate=0.2):
    population = initialize_population(population_size, num_stocks)
    for _ in range(generations):
        fitness_scores = [fitness(chromo, stock_returns, stock_risks) for chromo in population]
        new_population = []
        for _ in range(population_size // 2):
            parent1, parent2 = select(population, fitness_scores), select(population, fitness_scores)
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([mutate(child1, mutation_rate), mutate(child2, mutation_rate)])
        population = new_population
    best_index = np.argmax([fitness(chromo, stock_returns, stock_risks) for chromo in population])
    return population[best_index]

# 設定參數
start_date, end_date = "2020-01-01", "2023-01-01"
stock_symbols = ["2330", "1201", "2454", "2357", "4426", "8271"]
stock_returns, stock_risks = fetch_stock_data(stock_symbols, start_date, end_date)

# 執行演算法
best_portfolio = genetic_algorithm(stock_returns, stock_risks, len(stock_symbols))

# 輸出結果
print("最佳選股組合:", best_portfolio)
print("總報酬:", np.sum(best_portfolio * stock_returns))
print("總風險:", np.sqrt(np.sum((best_portfolio * stock_risks) ** 2)))
for i, symbol in enumerate(stock_symbols):
    print(f"{symbol}: {'選擇' if best_portfolio[i] else '不選擇'}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


最佳選股組合: [1 0 1 1 1 1]
總報酬: 1.4712498920044437
總風險: 3.1434678812664174
2330: 選擇
1201: 不選擇
2454: 選擇
2357: 選擇
4426: 選擇
8271: 選擇
